[참고] https://hansuho113.tistory.com/22
 - geopandas 설치 전 준비 사항   

In [1]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

# Url
xmlUrl = 'http://apis.data.go.kr/B553077/api/open/sdsc/storeZoneInAdmi'

#service Key
My_API_Key = unquote('')
queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('divId') : 'ctprvnCd',         #시도 
        quote_plus('key') : '30',                 #대전광역시 : 30
        quote_plus('ServiceKey') : My_API_Key     #인증키
    }
)

#get, encode
response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

In [2]:
xmlobj # url을 들어가면 대전에 대해서 나오는데 왜 python으로하면 전국이 나오는가.

<?xml version="1.0" encoding="utf-8"?>
<response>
<header>
<description>소상공인시장진흥공단 주요상권</description>
<resultCode>00</resultCode>
<resultMsg>NORMAL SERVICE</resultMsg>
</header>
<body>
<items>
<item>
<trarNo>1245</trarNo>
<mainTrarNm>대전 유성구 노은2동_2</mainTrarNm>
<ctprvnCd>30</ctprvnCd>
<ctprvnNm>대전광역시</ctprvnNm>
<signguCd>30200</signguCd>
<signguNm>유성구</signguNm>
<trarArea>108572.9</trarArea>
<coordNum>11</coordNum>
<coords>POLYGON ((127.320617 36.374723, 127.320624 36.375866, 127.318006 36.375866, 127.317991 36.37228, 127.320603 36.372271, 127.320613 36.374062, 127.322485 36.374059, 127.322524 36.374919, 127.321167 36.374922, 127.321167 36.374723, 127.320617 36.374723))</coords>
<stdrDt>2021-03-31</stdrDt>
</item>
<item>
<trarNo>1246</trarNo>
<mainTrarNm>대전 유성구 노은2동_1</mainTrarNm>
<ctprvnCd>30</ctprvnCd>
<ctprvnNm>대전광역시</ctprvnNm>
<signguCd>30200</signguCd>
<signguNm>유성구</signguNm>
<trarArea>83854.5</trarArea>
<coordNum>11</coordNum>
<coords>POLYGON ((127.313995 36.392837, 127.31325 36.

In [3]:
rows = xmlobj.findAll('item')

In [4]:
rows

[<item>
 <trarNo>1245</trarNo>
 <mainTrarNm>대전 유성구 노은2동_2</mainTrarNm>
 <ctprvnCd>30</ctprvnCd>
 <ctprvnNm>대전광역시</ctprvnNm>
 <signguCd>30200</signguCd>
 <signguNm>유성구</signguNm>
 <trarArea>108572.9</trarArea>
 <coordNum>11</coordNum>
 <coords>POLYGON ((127.320617 36.374723, 127.320624 36.375866, 127.318006 36.375866, 127.317991 36.37228, 127.320603 36.372271, 127.320613 36.374062, 127.322485 36.374059, 127.322524 36.374919, 127.321167 36.374922, 127.321167 36.374723, 127.320617 36.374723))</coords>
 <stdrDt>2021-03-31</stdrDt>
 </item>,
 <item>
 <trarNo>1246</trarNo>
 <mainTrarNm>대전 유성구 노은2동_1</mainTrarNm>
 <ctprvnCd>30</ctprvnCd>
 <ctprvnNm>대전광역시</ctprvnNm>
 <signguCd>30200</signguCd>
 <signguNm>유성구</signguNm>
 <trarArea>83854.5</trarArea>
 <coordNum>11</coordNum>
 <coords>POLYGON ((127.313995 36.392837, 127.31325 36.393842, 127.311247 36.393425, 127.311683 36.392075, 127.313085 36.392372, 127.313208 36.392291, 127.31312 36.392192, 127.31174 36.391895, 127.312212 36.390258, 127.315416

In [5]:
#리스트에 담기
columns = rows[0].find_all()

In [6]:
columns

[<trarNo>1245</trarNo>,
 <mainTrarNm>대전 유성구 노은2동_2</mainTrarNm>,
 <ctprvnCd>30</ctprvnCd>,
 <ctprvnNm>대전광역시</ctprvnNm>,
 <signguCd>30200</signguCd>,
 <signguNm>유성구</signguNm>,
 <trarArea>108572.9</trarArea>,
 <coordNum>11</coordNum>,
 <coords>POLYGON ((127.320617 36.374723, 127.320624 36.375866, 127.318006 36.375866, 127.317991 36.37228, 127.320603 36.372271, 127.320613 36.374062, 127.322485 36.374059, 127.322524 36.374919, 127.321167 36.374922, 127.321167 36.374723, 127.320617 36.374723))</coords>,
 <stdrDt>2021-03-31</stdrDt>]

In [7]:
# 행, 열 매트리스로 만들기
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없음)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행을 저장해야함    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워줌 
    
result = pd.DataFrame(rowList, columns=nameList)
result.head()

,trarNo,mainTrarNm,ctprvnCd,ctprvnNm,signguCd,signguNm,trarArea,coordNum,coords,stdrDt
0,1245,대전 유성구 노은2동_2,30,대전광역시,30200,유성구,108572.9,11,"POLYGON ((127.320617 36.374723, 127.320624 36....",2021-03-31
1,1246,대전 유성구 노은2동_1,30,대전광역시,30200,유성구,83854.5,11,"POLYGON ((127.313995 36.392837, 127.31325 36.3...",2021-03-31
2,1250,대전 유성구 관평동_1,30,대전광역시,30200,유성구,43785.5,12,"POLYGON ((127.390717 36.42566, 127.389864 36.4...",2021-03-31
3,1251,대전 유성구 구즉동_1,30,대전광역시,30200,유성구,36282,17,"POLYGON ((127.386795 36.435771, 127.386196 36....",2021-03-31
4,1233,대전 유성구 온천1동_1,30,대전광역시,30200,유성구,105516.5,34,"POLYGON ((127.335134 36.354419, 127.335494 36....",2021-03-31


In [8]:
result.shape

(39, 10)

- trarNo : 상권번호
- mainTrarNm : 상권명
- ctprvnCd : 시도(대전 : 30)
- signguCd : 시군구코드
- signguNm : 시군구명
- trarArea : 면적
- coordNum : 좌표개수
- coords : 좌표값
- stdrDt : 데이터기준일자


In [9]:
result.rename(columns = {"coords": "geometry"}, inplace = True)

In [10]:
result.head()

,trarNo,mainTrarNm,ctprvnCd,ctprvnNm,signguCd,signguNm,trarArea,coordNum,geometry,stdrDt
0,1245,대전 유성구 노은2동_2,30,대전광역시,30200,유성구,108572.9,11,"POLYGON ((127.320617 36.374723, 127.320624 36....",2021-03-31
1,1246,대전 유성구 노은2동_1,30,대전광역시,30200,유성구,83854.5,11,"POLYGON ((127.313995 36.392837, 127.31325 36.3...",2021-03-31
2,1250,대전 유성구 관평동_1,30,대전광역시,30200,유성구,43785.5,12,"POLYGON ((127.390717 36.42566, 127.389864 36.4...",2021-03-31
3,1251,대전 유성구 구즉동_1,30,대전광역시,30200,유성구,36282,17,"POLYGON ((127.386795 36.435771, 127.386196 36....",2021-03-31
4,1233,대전 유성구 온천1동_1,30,대전광역시,30200,유성구,105516.5,34,"POLYGON ((127.335134 36.354419, 127.335494 36....",2021-03-31


In [11]:
import geopandas as gpd

In [12]:
#변환 데이터 geojson으로 저장
result_geo = gpd.GeoDataFrame(result, geometry='geometry')
result_geo.to_file('D:/python/result_geo.geojson', driver='GeoJSON')

TypeError: Input must be valid geometry objects: POLYGON ((127.320617 36.374723, 127.320624 36.375866, 127.318006 36.375866, 127.317991 36.37228, 127.320603 36.372271, 127.320613 36.374062, 127.322485 36.374059, 127.322524 36.374919, 127.321167 36.374922, 127.321167 36.374723, 127.320617 36.374723))

### 문제 해결

- https://pythonq.com/so/python/1658831
 - 다각형의 형식은 WKT이므로 모양 다각형으로 변환해야합니다. 팔로 잉 Geopandas docs 

In [13]:
from shapely import wkt

result['geometry'] = result['geometry'].apply(wkt.loads)
result_geo = gpd.GeoDataFrame(result, geometry='geometry')

In [14]:
#geo_json으로 변환 
result_geo.to_file('D:/python/result_geo.geojson', driver='GeoJSON')

In [15]:
type(result_geo)

geopandas.geodataframe.GeoDataFrame

In [16]:
result_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   trarNo      39 non-null     object  
 1   mainTrarNm  39 non-null     object  
 2   ctprvnCd    39 non-null     object  
 3   ctprvnNm    39 non-null     object  
 4   signguCd    39 non-null     object  
 5   signguNm    39 non-null     object  
 6   trarArea    39 non-null     object  
 7   coordNum    39 non-null     object  
 8   geometry    39 non-null     geometry
 9   stdrDt      39 non-null     object  
dtypes: geometry(1), object(9)
memory usage: 3.2+ KB
